# TF-IDF e busca por conteúdo

Nesta atividade, vamos lidar com a seguinte situação: temos um grande banco de dados com textos, e queremos encontrar qual texto é mais relevante para uma consulta. Esse problema aparece em buscadores como Google, e também em sistemas locais como ElasticSearch.

In [30]:
import pandas as pd

DATASET = 'datasets/wikipedia_movies.zip'
df = pd.read_csv(DATASET).sample(1000)
df = df[['Title', 'Plot']]
print(df.head(), len(df))

                            Title  \
14935               Goya's Ghosts   
1663           Cockeyed Cavaliers   
33248                 Kids Return   
13552              Speedway Junky   
15298  Normal Adolescent Behavior   

                                                    Plot  
14935  In 1792, Spain reels amid the turmoil and uphe...  
1663   In medieval England, Bert and his friend, Bob ...  
33248  The movie is about two high school dropouts, M...  
13552  The film stars Jesse Bradford as Johnny, a you...  
15298  Wendy, Billie, and Ann are seniors at an alter...   1000


## Exercício 1
**Objetivo: lembrar-se do que é TF e o que é DF**

Identifique o Term Frequency e o Document Frequency nas asserções abaixo:

1. Quanto maior o ___, mais comum é a palavra entre os documentos de uma coleção
1. Quanto maior o ___, mais vezes a palavra é mencionada num documento específico
1. $P(w | \text{documento})$
1. $P(w | \text{coleção})$
1. Ajuda a identificar a coleção da qual um documento faz parte
1. Ajuda a identificar um documento dentro de uma coleção

## Exercício 2
**Objetivo: refletir sobre o uso de TF-IDF**

A medida TFIDF diz o quão relevante um documento é dentro de uma coleção e em relação a uma palavra específica. Ela é calculada para um par palavra-documento como:

$\text{TFIDF = TF / DF}$

Quando um documento tem um TFIDF alto em relação a uma palavra, isso significa que:

1. A palavra tende a ser (comum / incomum)
1. O documento menciona a palavras (muitas / poucas) vezes

Portanto, qual seria uma maneira de escrever um documento que tem intencionalmente um TFIDF alto para uma palavra?

## Exercício 3
**Objetivo: calcular TFIDF para documentos usando sklearn**

TFIDF pode ser entendido como um processo de vetorização, semelhante a usar o CountVectorizer. Abaixo, há um código que mostra um exemplo dessa vetorização usando sklearn. 

1. Escolhendo um filme aleatório da coleção que carregamos, identifique o TFIDF das palavras "zombie", "fungus" e "survival".
1. Identifique o filme que tem o maior TFIDF para a palavra "zombie".

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(df['Plot'])

j = vectorizer.vocabulary_['test']
print(tfidf[2,j])

0.0


In [32]:
# get a random movie
i = np.random.randint(0, len(df))
random_movie = df.iloc[i]['Title']

In [33]:
# for this random_movie, get the tfidf value for the word 'test'
zombie_vocabulary = vectorizer.vocabulary_['zombie']
# fungus_vocabulary = vectorizer.vocabulary_['fungus']
survival_vocabulary = vectorizer.vocabulary_['survival']
print(tfidf[i,zombie_vocabulary])
# print(tfidf[i,fungus_vocabulary])
print(tfidf[i,survival_vocabulary])

0.0
0.0


In [34]:
# for this random movie, get the tfidf of the word 'test'
j = vectorizer.vocabulary_['test']
print(tfidf[i,j])


0.0


In [40]:
# to the word Zombie, get the movie that has the highest tfidf
j = vectorizer.vocabulary_['zombie']
m = tfidf[:,j].argmax()
print(tfidf[:,j].toarray().argmax())
print("tfidf: ", tfidf[m,j])
print(df.iloc[tfidf[:,j].toarray().argmax()]['Title'])

30
tfidf:  0.16149123461212997
Warm Bodies


In [26]:
# get the row number of Fleshheater
print(df[df['Title'] == 'Flesheater'])

            Title                                               Plot
10954  Flesheater  The film starts with a group of kids taking a ...


In [12]:
print(df.iloc[tfidf[:,j].toarray().argmax()]['Plot'])

The film starts with a group of kids taking a hayride in the country on Halloween. They pay the local farmer to take them to a secluded area of the forest. The kids arrive and begin drinking, telling the farmer to come back after dark to pick them up. As the party wears on the group separates to find their own little love nests.
Meanwhile, the farmer has stumbled across a large tree stump which he proceeds to remove with the help of his tractor. Under the stump is a large wooden box with an ancient seal telling not to break open the box. The farmer breaks the seal and opens the box. Inside is the Bill Heinzman "Flesheater" who precedes to eat the farmer making him a zombie in the process. Both zombies head towards where the kids are.
Two of the kids who retreated to the barn for some alone time are killed by the flesheaters. As the flesh-eater is killing the kids, two of their friends walk in and see what's happening then they run outside to warn the group at the party. Inside the barn

## Exercício 4
**Objetivo: implementar uma busca por vários termos simultaneamente**

Uma possível maneira de implementar uma busca por vários termos é somar o TFIDF de todas as palavras da query para cada documento da coleção, e então retornar o documento que tem a maior soma. Por exemplo, numa busca por "zombie fungus survival" deveríamos somar, para cada documento, o TFIDF de "zombie", de "fungus" e de "survival" e então ordenar o resultado.

1. Escreva código que implemente uma busca na base de dados de filmes à partir de uma query específica.
1. Qual é a complexidade ($O(...)$) da sua busca?

In [49]:
import re
query = "love robot"

# Implemente sua solução aqui
query_words = re.findall(r'\w+', query)

# for each word in the query, get the tfidf value
vectorizer_list = []
for word in query_words:
    if word not in vectorizer.vocabulary_:
        continue
    j = vectorizer.vocabulary_[word]
    vectorizer_list.append(j)
# 
local_tfidf = tfidf[:,vectorizer_list]

sum_local_tfidf = np.sum(local_tfidf, axis=1)

# get the movie with the highest tfidf
m = sum_local_tfidf.argmax()
print(sum_local_tfidf.argmax())
print("tfidf: ", sum_local_tfidf[m])
print(df.iloc[sum_local_tfidf.argmax()]['Title'])

860
tfidf:  [[0.22600513]]
Killing 'em Softly


## Exercício 5
**Objetivo: implementar um índice invertido**

Você provavelmente reparou (talvez não tenha reparado, e é tudo bem) que, para fazer a busca, até agora, teve que varrer todos os documentos da sua coleção. Isso provavelmente levaria algum tempo, especialmente quando a coleção começa a aumentar. Para evitar ter que varrer todos os documentos da coleção, podemos implementar uma técnica chamada *índice invertido*. A ideia do índice invertido é usar um dicionário cujas chaves são as palavras do vocabulário e cujo conteúdo é uma lista de documentos que contém essa palavra, possivelmente acompanhados do TFIDF correspondente. Por exemplo:

In [54]:
indice = { 'palavra_1' : {'documento_1': 0.5, 'documento_2': 0.1}, 
          'palavra_2' : {'documento_2': 0.6},
            'equalization': {'documento_1': 0.5}
}

def buscar(palavras, indice):
    assert type(palavras)==list
    resultado = dict()
    for p in palavras:
        if p in indice.keys():
            for documento in indice[p].keys():
                if documento not in resultado.keys():
                    resultado[documento] = indice[p][documento]
                else:
                    resultado[documento] += indice[p][documento]
    return resultado

buscar(['palavra_1', 'palavra_2','equalization'], indice)


def n_maiores(res_busca,n):
    res = []
    for k in res_busca.keys():
        res.append((k,res_busca[k]))
    res = sorted(res, key=lambda x: x[1], reverse=True)
    return res[:n]

res = buscar(['palavra_1', 'palavra_2'], indice)

print(n_maiores(res,2))

def query(query, indice, n):
    palavras = re.findall(r'\w+', query)
    res_busca = buscar(palavras, indice)
    return n_maiores(res_busca,n)

[('documento_2', 0.7), ('documento_1', 0.5)]


In [55]:
query('palavra_1 palavra_2', indice, 2)

[('documento_2', 0.7), ('documento_1', 0.5)]

1. Adicione uma nova palavra ao índice e escolha seu TFIDF. Realize uma nova busca e verifique o resultado.
1. Escreva uma função que ordena o resultado e retorna apenas `N` documentos mais relevantes para sua busca.
1. Incremente sua biblioteca de forma que ela passe a receber uma string como entrada (representando a query) e retorne os `N` documentos mais relevantes (`N` pode ser definido arbitrariamente).

## Exercício 6
**Objetivo: implementar um buscador de filmes**

Implemente uma função que recebe como entrada uma query e retorna os títulos e enredos dos 5 filmes mais relevantes para aquela query. Se precisar, use mais parâmetros ou variáveis globais. Teste a sua função e veja se você concorda com os resultados, incluindo se você consegue encontrar seus filmes favoritos e se consegue alguma recomendação relevante a um filme novo.

In [34]:
def query_movies(query : str):
    pass

## Exercício 7
**Objetivo: identificar palavras-chave usando TFIDF**

Uma outra aplicação de TFIDF é encontrar palavras-chave, isto é, palavras que diferenciam um documento do restante dos documentos de sua coleção.

Incremente seu buscador de forma que, além do título e enredo, ele também escolha as algumas palavras (escolha quantas!) mais relevantes de cada documento e as imprima como keywords.

## Exercício 8
**Objetivo: encontrar documentos semelhantes usando TFIDF**

Uma maneira de encontrar documentos semelhantes em uma coleção de textos é assumir que o texto do documento é uma query, e então realizar a busca normalmente. O problema disso é que provavelmente teríamos textos muito longos e a query ficaria muito carregada. Para solucionar isso, poderíamos usar apenas as palavras mais relevantes de um documento como query. Implemente uma função que recebe o índice (ou outro identificador único) de um documento de nosso banco de dados e então encontra 5 documentos semelhantes a ele.